## Dataset generated by traditional ECFPs

In [2]:

import numpy as np
import pandas as pd
from pathlib import Path

path_to_scripts = '../../scripts'
sys.path.append(path_to_scripts)
from smiles_property_extractor import Fingerprint


In [3]:

db = pd.read_csv('../../datasets/processed/cleaned_data_v1.csv', sep=',')


In [4]:

print('checking duplications based on number of SMILES ...')
#DB['Temperature']
db[db.duplicated(subset=['SMILES'])]


checking duplications based on number of SMILES ...


,molindx,Name,SMILES,MolarMass,CASNumber,ExperimentalSolubilityInWater,Temperature,ExperimentReference,Datagroup,SolvationEnergy,...,CalMR,TPSA,LabuteASA,BalabanJIndex,BertzCTIndex,Isomer,Aromatic,Cyclic,ChargeNeutral,MadeOfKeyElements


In [5]:

print('Generating a new database :')
db_ = db[['molindx', 'SMILES', 'Temperature', 'ExperimentalSolubilityInWater']]

db_ = db_.assign(logS=lambda x: (np.log(x['ExperimentalSolubilityInWater'])))

db_['logS'] = db_['logS'].round(5)

db_['Temperature'] = db_['Temperature'] / max(db_['Temperature'])

db_new = db_.drop("ExperimentalSolubilityInWater", axis='columns')
db_new


Generating a new database :


,molindx,SMILES,Temperature,logS
0,0,C1(=NC(=NC(=N1)Cl)Cl)Cl,0.635511,10.81559
1,1,C1=C(C=C(C=C1[N+](=O)[O-])[N+](=O)[O-])[N+](=O...,0.614196,5.62659
2,2,C1=C(C=C(C(=C1[N+](=O)[O-])C(=O)O)[N+](=O)[O-]...,0.633380,9.92904
3,3,C1=C(C=C(C=C1[N+](=O)[O-])[N+](=O)[O-])C(=O)O,0.635511,7.20786
4,4,[O-][N+](=O)C1=CC(=C(Cl)C(=C1)[N+]([O-])=O)[N+...,0.616327,6.27377
...,...,...,...,...
4630,4630,CC1=CC(=O)N=C(N1)S,0.635511,6.27897
4631,4631,COP(=S)(OC)SCC1=NC(=NC(=N1)N)N,0.624853,5.48050
4632,4632,CC(=O)N[C@@H](CC1=CC(=C(C(=C1)Br)O)Br)C(=O)O,0.635511,7.82406
4633,4633,CN1/C(=C(\NC2=CC=CC=N2)/O)/C(=O)C3=CC=CC=C3S1(...,0.629116,3.13549


In [ ]:

DATA_PATH = "../../datasets/processed/"
diameter = 6
nbits=1024
is_numbering_fragments=False

f = Fingerprint(db_, DATA_PATH, is_numbering_fragments)
if Path('%s/fingerprints.csv' % DATA_PATH).exists():
    print('Reading features from file...')
    xbits = pd.read_csv('%s/fingerprints.csv' % DATA_PATH)
else:
    print('Computing features from scratch... THIS FAILS!! ')
    f.generate_ecfp(nbits=nbits, diameter=diameter)      # 1024
    xbits = pd.DataFrame(f.fingerprints)


Computing features from scratch... THIS FAILS!! 


Processing fingerprints:  80%|█████████████████████████████████████████▍          | 3696/4635 [01:59<00:43, 21.45it/s]

In [ ]:

xbits[xbits.duplicated()]


In [ ]:

db_org = db_.drop(columns=['SMILES', 'ExperimentalSolubilityInWater'], axis=1)
db_org


In [ ]:

data = pd.concat([xbits, db_org], axis=1)
data


In [ ]:

data.to_csv('../../datasets/processed/dataset-ECFPDescriptors.csv', index=False)
